In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import json
import time
import datetime
import random
import bisect
import itertools

In [2]:
def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f) 
        _attrs = dict(source='sender', target='receiver', name='guid',
              key='guid', link='links')
    return nx.readwrite.node_link_graph(js_graph, directed=True, multigraph=False, attrs={'link': 'links', 'source': 'sender', 'target': 'receiver', 'key': 'guid', 'name': 'guid'} )

G = read_json_file("data.json")

In [3]:
def WIS(graph, size=-1, weights=None):   # weighted    
    print(nx.info(graph))
    
    n = graph.number_of_nodes()
    print("graph.number_of_nodes()", graph.number_of_nodes())
    cum_weights = [0]*n
    tot_weight = 0
    index_count = 0
    integers = []
    nodes = [0]*n
    node_indexes = []
    node_prob = [] 
 
    for i,v in enumerate(list(G.nodes(data=True))): 
        tot_weight += graph.degree(v[0], weight='lastTs') 
        cum_weights[i] = graph.degree(v[0], weight='lastTs')
        node_indexes.append(i)
        nodes[i] = v[0]
        node_prob.append(graph.degree(v[0], weight='lastTs'))  
    
    node_prob[:] = [x / tot_weight for x in node_prob]  
     
    c = 0 
    while(c < size):
        if c>size: return  
        i = np.random.choice(node_indexes, p=node_prob) 
        if nodes[i] not in integers:
            c = c + 1 
            integers.append(nodes[i])  
    
    return integers

In [5]:
size = int(G.number_of_nodes() / 2)
G_r = G.copy()
sample = WIS(G_r, size=size, weights=G.degree)
 
nodes_to_remove = [x for x in G_r.nodes if x not in sample] 
G_r.remove_nodes_from(nodes_to_remove)
print(nx.info(G_r))

Name: 
Type: DiGraph
Number of nodes: 6489
Number of edges: 52318
Average in degree:   8.0626
Average out degree:   8.0626
graph.number_of_nodes() 6489
Name: 
Type: DiGraph
Number of nodes: 3244
Number of edges: 35921
Average in degree:  11.0731
Average out degree:  11.0731


In [6]:
# save reduced graph into a file
_attrs = dict(source='sender', target='receiver', name='guid',
              key='guid', link='links')
s2 = nx.readwrite.node_link_data(G_r, attrs={'link': 'links', 'source': 'sender', 'target': 'receiver', 'key': 'guid', 'name': 'guid'})

with open('data_processed_WIS.json', 'w') as outfile:
    json.dump(s2, outfile)
